In [1]:
import re
from hashlib import sha256
import requests
import datetime
from pathlib import Path

from fetchers.FetchConfig import FetchConfig
from fetchers import DiscoverDownloadVerifyUnpackFetcher, GitRepoFetcher
from fetchers.discoverers.GitHub import GitHubReleasesDiscoverer
from fetchers.discoverers.DNF import DNFDiscoverer
from fetchers.unpackers import GZIP, Archive, Tar
from fetchers.verifiers import GPGVerifier, HashsesVerifier

thisDir = Path(".").absolute()
cfg = FetchConfig(downloadsTmp=thisDir / "downloads")

# Downloading source code from a Git repo

In [2]:
moduleMdFetcher = GitRepoFetcher("https://github.com/fedora-modularity/libmodulemd", refspec="main")
fetchedInfo = moduleMdFetcher(thisDir / "modulemd", None)  # for this one config is not necessary

Widget Javascript not detected.  It may not be installed or enabled properly.


Detected service <fetchers.webServices.services.GitHub.GitHubService object at 0x000000000000>


50 / 60 83.33333333333334% limit will be reset: 2020-06-29 00:00:00 in -1 day, 00:00:00.000000
Pulling https://github.com/fedora-modularity/libmodulemd to ./modulemd ...


Pulled



version from tag: 2.9.4


In [3]:
print(repr(fetchedInfo.version))
print(fetchedInfo.version)
print(fetchedInfo.version.major)
print(fetchedInfo.version.minor)
print(fetchedInfo.version.patch)

AnyVer('2.9.4')
2.9.4
2
9
4


In [4]:
datetime.datetime.fromtimestamp(fetchedInfo.time)

datetime.datetime(2020, 6, 22, 18, 56, 10)

# Downloading binary builds from GitHub releases
(Tar unpacker has a minor bug in progress reporter: it overestimates the size of files in the tarball a bit, that's why the progressbar is red)

In [5]:
# Getting Bradley King's key (you should ship the key, I just don't want to junk the repo, but the lib can also accept `bytes`) and saving it
bradKeyResp = requests.get("https://keyserver.ubuntu.com/pks/lookup?op=get&search=0xcba23971357c2e6590d9efd3ec8fef3a7bfb4eda")
bradKeyPath = thisDir / "brad.gpg"
bradKeyPath.write_bytes(bradKeyResp.content)

35911

In [6]:
versionRxText = "(?:\\d+\\.){1,2}\\d+(?:-rc\\d+)?"
platformMarker = "Linux-x86_64"
hashFuncName = "SHA-256"
downloadFileNameRx = re.compile("^" + "-".join(("cmake", versionRxText, platformMarker)) + "\\.tar\\.gz$")
hashesFileNameRxText = "-".join(("cmake", versionRxText, hashFuncName)) + "\\.txt"
hashesSigFileNameRxText = hashesFileNameRxText + "\\.(?:asc|sig|gpg)"

cmakeFetcher = DiscoverDownloadVerifyUnpackFetcher(
    GithubReleasesDiscoverer(
        "Kitware/CMake", tagRx=re.compile("^v(" + versionRxText + ")$"),
        downloadFileNamesRxs={
            "binary.tgz": downloadFileNameRx,
            "hashes": re.compile("^" + hashesFileNameRxText + "$"),
            "hashes.asc": re.compile("^" + hashesSigFileNameRxText + "$"),
        },
        titleRx = None
    ),
    HashsesVerifier(GPGVerifier, sha256, hashesFileName="hashes",  signatureFileName="hashes.asc", keyFingerprint = "CBA23971357C2E6590D9EFD3EC8FEF3A7BFB4EDA", keyFile=bradKeyPath),
    Archive("binary.tgz", Tar, GZIP)
)
fetchedInfo = cmakeFetcher("./cmake", cfg)

49 / 60 81.66666666666667% limit will be reset: 2020-06-29 00:00:00 in -1 day, 00:00:00.000000


Selected release:  [3.18.0-rc2, prerelease] <{'binary.tgz': DownloadTargetFile('binary.tgz', 'cmake-3.18.0-rc2-Linux-x86_64.tar.gz', datetime.datetime(2020, 6, 17, 16, 33, 17, tzinfo=tzutc()), datetime.datetime(2020, 6, 17, 16, 33, 49, tzinfo=tzutc()), 'https://github.com/Kitware/CMake/releases/download/v3.18.0-rc2/cmake-3.18.0-rc2-Linux-x86_64.tar.gz', 41690438), 'hashes': DownloadTargetFile('hashes', 'cmake-3.18.0-rc2-SHA-256.txt', datetime.datetime(2020, 6, 17, 16, 32, 48, tzinfo=tzutc()), datetime.datetime(2020, 6, 17, 16, 32, 48, tzinfo=tzutc()), 'https://github.com/Kitware/CMake/releases/download/v3.18.0-rc2/cmake-3.18.0-rc2-SHA-256.txt', 972), 'hashes.asc': DownloadTargetFile('hashes.asc', 'cmake-3.18.0-rc2-SHA-256.txt.asc', datetime.datetime(2020, 6, 17, 16, 31, 46, tzinfo=tzutc()), datetime.datetime(2020, 6, 17, 16, 31, 46, tzinfo=tzutc()), 'https://github.com/Kitware/CMake/releases/download/v3.18.0-rc2/cmake-3.18.0-rc2-SHA-256.txt.asc', 833)}>


downloadedTargets {'binary.tgz': DownloadTarget('binary.tgz', 'cmake-3.18.0-rc2-Linux-x86_64.tar.gz', 'https://github.com/Kitware/CMake/releases/download/v3.18.0-rc2/cmake-3.18.0-rc2-Linux-x86_64.tar.gz', PosixPath('./downloads/binary.tgz'), datetime.datetime(2020, 6, 17, 16, 33, 49, tzinfo=tzutc())), 'hashes': DownloadTarget('hashes', 'cmake-3.18.0-rc2-SHA-256.txt', 'https://github.com/Kitware/CMake/releases/download/v3.18.0-rc2/cmake-3.18.0-rc2-SHA-256.txt', PosixPath('./downloads/hashes'), datetime.datetime(2020, 6, 17, 16, 32, 48, tzinfo=tzutc())), 'hashes.asc': DownloadTarget('hashes.asc', 'cmake-3.18.0-rc2-SHA-256.txt.asc', 'https://github.com/Kitware/CMake/releases/download/v3.18.0-rc2/cmake-3.18.0-rc2-SHA-256.txt.asc', PosixPath('./downloads/hashes.asc'), datetime.datetime(2020, 6, 17, 16, 31, 46, tzinfo=tzutc()))}
hashes {'cmake-3.18.0-rc2-Darwin-x86_64.dmg': 'f68d91a0a44ff5ebf0d0507d6d4b6db7c20f7674a16c76105973635867828314', 'cmake-3.18.0-rc2-Darwin-x86_64.tar.gz': 'fe46e0405

Widget Javascript not detected.  It may not be installed or enabled properly.


In [7]:
fetchedInfo.version

AnyVer('3.18.0-rc2')

In [8]:
fetchedInfo.time

datetime.datetime(2020, 6, 17, 16, 33, 49, tzinfo=tzutc())